In [1]:
# !pip install /kaggle/input/staintools-offline/spams-2.6.5.4-cp37-cp37m-linux_x86_64.whl
# !pip install /kaggle/input/staintools-offline/staintools-2.1.2-py3-none-any.whl

In [2]:
# import staintools

# def get_target(organ_type):
#     if organ_type == "kidney":
#         target = staintools.read_image("/kaggle/input/hubmap-organ-segmentation/train_images/15005.tiff")
#         return target
#     elif organ_type == "largeintestine":
#         target = staintools.read_image("/kaggle/input/hubmap-organ-segmentation/train_images/9777.tiff")
#         return target
#     elif organ_type == "lung":
#         target = staintools.read_image("/kaggle/input/hubmap-organ-segmentation/train_images/4301.tiff")
#         return target
#     elif organ_type == "prostate":
#         target = staintools.read_image("/kaggle/input/hubmap-organ-segmentation/train_images/12244.tiff")
#         return target
#     elif organ_type == "spleen":
#         target = staintools.read_image("/kaggle/input/hubmap-organ-segmentation/train_images/1123.tiff")
#         return target

In [3]:
# import staintools
# target_kidney = staintools.read_image("/kaggle/input/hubmap-organ-segmentation/train_images/15005.tiff")
# target_largeintestine = staintools.read_image("/kaggle/input/hubmap-organ-segmentation/train_images/9777.tiff")
# target_lung = staintools.read_image("/kaggle/input/hubmap-organ-segmentation/train_images/4301.tiff")
# target_prostate = staintools.read_image("/kaggle/input/hubmap-organ-segmentation/train_images/12244.tiff")
# target_spleen = staintools.read_image("/kaggle/input/hubmap-organ-segmentation/train_images/1123.tiff")
# print('速度')

In [4]:
import os
import cv2
import matplotlib.pyplot as plt
import time
import random

import torch
from torch import nn
import torch.cuda.amp as amp
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import RandomSampler 
from torch.utils.data import SequentialSampler
import torch.nn.functional as F
from torchmetrics.functional import dice_score
from torch.optim.lr_scheduler import StepLR
import tifffile
from fastai.vision.all import *

from collections import defaultdict

import torch
from torch.optim.optimizer import Optimizer
import itertools as it

is_amp = True
import logging
import pandas as pd
from sklearn.model_selection import KFold

import numpy as np
from itertools import repeat
import collections.abc
import math
import torch
from torch.optim.optimizer import Optimizer, required
from collections import defaultdict

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

sys.path.append('../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master')

from efficientnet_pytorch import EfficientNet
from efficientnet_pytorch.utils import url_map, url_map_advprop, get_model_params

In [5]:
class FPN(nn.Module):
    def __init__(self, input_channels:list, output_channels:list):
        super().__init__()
        self.convs = nn.ModuleList(
            [nn.Sequential(nn.Conv2d(in_ch, out_ch*2, kernel_size=3, padding=1),
             nn.ReLU(inplace=True), nn.BatchNorm2d(out_ch*2),
             nn.Conv2d(out_ch*2, out_ch, kernel_size=3, padding=1))
            for in_ch, out_ch in zip(input_channels, output_channels)])
        
    def forward(self, xs:list, last_layer):
        hcs = [F.interpolate(c(x),scale_factor=2**(len(self.convs)-i),mode='bilinear') 
               for i,(c,x) in enumerate(zip(self.convs, xs))]
        hcs.append(last_layer)
        return torch.cat(hcs, dim=1)

class UnetBlock(nn.Module):
    def __init__(self, up_in_c:int, x_in_c:int, nf:int=None, blur:bool=False,
                 self_attention:bool=False, **kwargs):
        super().__init__()
        self.shuf = PixelShuffle_ICNR(up_in_c, up_in_c//2, blur=blur, **kwargs)
        self.bn = nn.BatchNorm2d(x_in_c)
        ni = up_in_c//2 + x_in_c
        nf = nf if nf is not None else max(up_in_c//2,32)
        self.conv1 = ConvLayer(ni, nf, norm_type=None, **kwargs)
        self.conv2 = ConvLayer(nf, nf, norm_type=None,
            xtra=SelfAttention(nf) if self_attention else None, **kwargs)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, up_in:Tensor, left_in:Tensor) -> Tensor:
        s = left_in
        up_out = self.shuf(up_in)
        cat_x = self.relu(torch.cat([up_out, self.bn(s)], dim=1))
        return self.conv2(self.conv1(cat_x))
        
class _ASPPModule(nn.Module):
    def __init__(self, inplanes, planes, kernel_size, padding, dilation, groups=1):
        super().__init__()
        self.atrous_conv = nn.Conv2d(inplanes, planes, kernel_size=kernel_size,
                stride=1, padding=padding, dilation=dilation, bias=False, groups=groups)
        self.bn = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU()

        self._init_weight()

    def forward(self, x):
        x = self.atrous_conv(x)
        x = self.bn(x)

        return self.relu(x)

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

class ASPP(nn.Module):
    def __init__(self, inplanes=512, mid_c=256, dilations=[6, 12, 18, 24], out_c=None):
        super().__init__()
        self.aspps = [_ASPPModule(inplanes, mid_c, 1, padding=0, dilation=1)] + \
            [_ASPPModule(inplanes, mid_c, 3, padding=d, dilation=d,groups=4) for d in dilations]
        self.aspps = nn.ModuleList(self.aspps)
        self.global_pool = nn.Sequential(nn.AdaptiveMaxPool2d((1, 1)),
                        nn.Conv2d(inplanes, mid_c, 1, stride=1, bias=False),
                        nn.BatchNorm2d(mid_c), nn.ReLU())
        out_c = out_c if out_c is not None else mid_c
        self.out_conv = nn.Sequential(nn.Conv2d(mid_c*(2+len(dilations)), out_c, 1, bias=False),
                                    nn.BatchNorm2d(out_c), nn.ReLU(inplace=True))
        self.conv1 = nn.Conv2d(mid_c*(2+len(dilations)), out_c, 1, bias=False)
        self._init_weight()

    def forward(self, x):
        x0 = self.global_pool(x)
        xs = [aspp(x) for aspp in self.aspps]
        x0 = F.interpolate(x0, size=xs[0].size()[2:], mode='bilinear', align_corners=True)
        x = torch.cat([x0] + xs, dim=1)
        return self.out_conv(x)
    
    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()


class config:
    pretrained_root = '../input/efficientnet-pytorch/'
    efficient_net_encoders = {
        "efficientnet-b0": {
            "out_channels": (3, 32, 24, 40, 112, 320),
            "stage_idxs": (3, 5, 9, 16),
            "weight_path": pretrained_root + "efficientnet-b0-08094119.pth"
        },
        "efficientnet-b1": {
            "out_channels": (3, 32, 24, 40, 112, 320),
            "stage_idxs": (5, 8, 16, 23),
            "weight_path": pretrained_root + "efficientnet-b1-dbc7070a.pth"
        },
        "efficientnet-b2": {
            "out_channels": (3, 32, 24, 48, 120, 352),
            "stage_idxs": (5, 8, 16, 23),
            "weight_path": pretrained_root + "efficientnet-b2-27687264.pth"
        },
        "efficientnet-b3": {
            "out_channels": (3, 40, 32, 48, 136, 384),
            "stage_idxs": (5, 8, 18, 26),
            "weight_path": pretrained_root + "efficientnet-b3-c8376fa2.pth"
        },
        "efficientnet-b4": {
            "out_channels": (3, 48, 32, 56, 160, 448),
            "stage_idxs": (6, 10, 22, 32),
            "weight_path": pretrained_root + "efficientnet-b4-e116e8b3.pth"
        },
        "efficientnet-b5": {
            "out_channels": (3, 48, 40, 64, 176, 512),
            "stage_idxs": (8, 13, 27, 39),
            "weight_path": pretrained_root + "efficientnet-b5-586e6cc6.pth"
        },
        "efficientnet-b6": {
            "out_channels": (3, 56, 40, 72, 200, 576),
            "stage_idxs": (9, 15, 31, 45),
            "weight_path": pretrained_root + "efficientnet-b6-c76e70fd.pth"
        },
        "efficientnet-b7": {
            "out_channels": (3, 64, 48, 80, 224, 640),
            "stage_idxs": (11, 18, 38, 55),
            "weight_path": pretrained_root + "efficientnet-b7-dcc49843.pth"
        }
    }
    model = 'efficientnet-b7'
    
class EfficientNetEncoder(EfficientNet):
    def __init__(self, stage_idxs, out_channels, model_name, depth=5):

        blocks_args, global_params = get_model_params(model_name, override_params=None)
        super().__init__(blocks_args, global_params)
        
        cfg = config.efficient_net_encoders[model_name]

        self._stage_idxs = stage_idxs
        self._out_channels = out_channels
        self._depth = depth
        self._in_channels = 3

        del self._fc
        self.load_state_dict(torch.load(cfg['weight_path']))

    def get_stages(self):
        return [
            nn.Identity(),
            nn.Sequential(self._conv_stem, self._bn0, self._swish),
            self._blocks[:self._stage_idxs[0]],
            self._blocks[self._stage_idxs[0]:self._stage_idxs[1]],
            self._blocks[self._stage_idxs[1]:self._stage_idxs[2]],
            self._blocks[self._stage_idxs[2]:],
        ]

    def forward(self, x):
        stages = self.get_stages()

        block_number = 0.
        drop_connect_rate = self._global_params.drop_connect_rate

        features = []
        for i in range(self._depth + 1):

            # Identity and Sequential stages
            if i < 2:
                x = stages[i](x)

            # Block stages need drop_connect rate
            else:
                for module in stages[i]:
                    drop_connect = drop_connect_rate * block_number / len(self._blocks)
                    block_number += 1.
                    x = module(x, drop_connect)

            features.append(x)

        return features

    def load_state_dict(self, state_dict, **kwargs):
        state_dict.pop("_fc.bias")
        state_dict.pop("_fc.weight")
        super().load_state_dict(state_dict, **kwargs)  
        

class EffUnet(nn.Module):
    def __init__(self, model_name, stride=1):
        super().__init__()
        
        cfg = config.efficient_net_encoders[model_name]
        stage_idxs = cfg['stage_idxs']
        out_channels = cfg['out_channels']
        
        self.encoder = EfficientNetEncoder(stage_idxs, out_channels, model_name)

        #aspp with customized dilatations
        self.aspp = ASPP(out_channels[-1], 256, out_c=384, 
                         dilations=[stride*1, stride*2, stride*3, stride*4])
        self.drop_aspp = nn.Dropout2d(0.5)
        #decoder
        self.dec4 = UnetBlock(384, out_channels[-2], 256)
        self.dec3 = UnetBlock(256, out_channels[-3], 128)
        self.dec2 = UnetBlock(128, out_channels[-4], 64)
        self.dec1 = UnetBlock(64, out_channels[-5], 32)
        self.fpn = FPN([384, 256, 128, 64], [16]*4)
        self.drop = nn.Dropout2d(0.1)
        self.final_conv = ConvLayer(32+16*4, 1, ks=1, norm_type=None, act_cls=None)
        
        self.rgb = RGB()
        
    def forward(self, batch):
        x = batch['image']
        B, C, H, W = x.shape
        x = self.rgb(x)
        enc0, enc1, enc2, enc3, enc4 = self.encoder(x)[-5:]
        enc5 = self.aspp(enc4)
        dec3 = self.dec4(self.drop_aspp(enc5), enc3)
        dec2 = self.dec3(dec3,enc2)
        dec1 = self.dec2(dec2,enc1)
        dec0 = self.dec1(dec1,enc0)
        x = self.fpn([enc5, dec3, dec2, dec1], dec0)
        x = self.final_conv(self.drop(x))
        x = F.interpolate(x, size = 512, mode = 'bilinear')
        return x

In [6]:
from tqdm.notebook import tqdm
import gc
import tifffile as tiff
import tifffile
import numpy as np
import matplotlib.pyplot as plt
from rasterio.windows import Window
import pandas as pd
import rasterio

is_amp = True
import warnings

warnings.filterwarnings('ignore')
import sys

sys.path.append("../input/deeplearnings")
from dataset import *
sys.path.append("../input/deeplearnings")
from imagecut import *
from imageplt import *
from model import *
from model_coat_daformer import *
from model_pvt_v2_daformer import *
from model_coat_daformer_2 import *
from My_model_daformer_coat import *
def rle2mask(mask_rle, shape=(1600, 256)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T


tensor_list = [
    'mask', 'image', 'organ',
]


def null_collate(batch):
    d = {}
    key = batch[0].keys()
    for k in key:
        v = [b[k] for b in batch]
        if k in tensor_list:
            v = torch.stack(v)
        d[k] = v
    # d['mask'] = d['mask'].unsqueeze(1)
    # d['organ'] = d['organ'].reshape(-1)
    return d


def image_to_tensor(image, mode='bgr'):  # image mode
    if mode == 'bgr':
        image = image[:, :, ::-1]
    x = image
    x = x.transpose(2, 0, 1)
    x = np.ascontiguousarray(x)
    x = torch.tensor(x, dtype=torch.float)
    return x


def mask_to_tensor(mask):
    x = mask
    x = torch.tensor(x, dtype=torch.float)
    return x


def tensor_to_mask(x):
    mask = x.data.cpu().numpy()
    mask = mask.astype(np.float32)
    return mask
imaging_measurements = {
    'hpa': {
        'pixel_size': {
            'kidney': 0.4,
            'prostate': 0.4,
            'largeintestine': 0.4,
            'spleen': 0.4,
            'lung': 0.4
        },
        'tissue_thickness': {
            'kidney': 4,
            'prostate': 4,
            'largeintestine': 4,
            'spleen': 4,
            'lung': 4
        }
    },
    'hubmap': {
        'pixel_size': {
            'kidney': 0.5,
            'prostate': 6.263,
            'largeintestine': 0.229,
            'spleen': 0.4945,
            'lung': 0.7562
        },
        'tissue_thickness': {
        'kidney': 10,
            'prostate': 5,
            'largeintestine': 8,
            'spleen': 4,
            'lung': 5
        }
    }
}


def pixel_size(image, domain_pixel_size, target_pixel_size):
    pixel_size_scale_factor = domain_pixel_size / target_pixel_size

    image_resized = cv2.resize(
        image,
        dsize=None,
        fx=pixel_size_scale_factor,
        fy=pixel_size_scale_factor,
        interpolation=cv2.INTER_CUBIC
    )

    image_resizedd = cv2.resize(
        image_resized,
        dsize=(
            image.shape[1],
            image.shape[0]
        ),
        interpolation=cv2.INTER_CUBIC
    )
    return image_resizedd


##数据处理
class HubmapDataset(Dataset):
    def __init__(self, df, augment=None,image_size=1024):
        data_path = "../input/hubmap-organ-segmentation"
        df['image_path'] = df['id'].apply(lambda x: os.path.join(data_path, 'test_images', str(x) + '.tiff'))
        self.df = df
        self.augment = augment
        self.length = len(self.df)
        self.image_size=image_size

    def __str__(self):
        string = ''
        string += '\tlen = %d\n' % len(self)

        d = self.df.organ.value_counts().to_dict()
        for k in ['kidney', 'prostate', 'largeintestine', 'spleen', 'lung']:
            string += '%24s %3d (%0.3f) \n' % (k, d.get(k, 0), d.get(k, 0) / len(self.df))
        return string

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        img_path = self.df.loc[index, 'image_path']
        image = tifffile.imread(img_path) 
        
#         domain_pixel_size = imaging_measurements['hpa']['pixel_size'][row['organ']]
#         target_pixel_size = imaging_measurements['hubmap']['pixel_size'][row['organ']]
#         image = pixel_size(image, domain_pixel_size, target_pixel_size)
        

#         target = get_target(self.df.organ.values[0])##添加-jion
#         image = staintools.LuminosityStandardizer.standardize(image)
        
#         # Stain normalize
#         normalizer = staintools.StainNormalizer(method='vahadane')
#         normalizer.fit(target)
#         transformed3 = normalizer.transform(image)
#         image = transformed3
        
        

        image = image.astype(np.float32) / 255

        #image_size = 1024
        if self.image_size>1024:
            image = cv2.resize(image, dsize=(1024, 1024), interpolation=cv2.INTER_LINEAR)
        else:
            image = cv2.resize(image, dsize=(self.image_size, self.image_size), interpolation=cv2.INTER_LINEAR)
        
        r = {}
        r['index'] = index
        r['id'] = self.df['id'][0]
        r['image'] = image_to_tensor(image)
        return r


############################################################
####### Validation
############################################################
def validate(net, valid_loader):
    net = net.eval()
    image_size = int(valid_loader.dataset.df['img_height'][0])
    for t, batch in enumerate(valid_loader):
        net.output_type = ['inference']
        with torch.no_grad():
            with amp.autocast(enabled=is_amp):
                batch['image'] = batch['image'].to(device)
                output = net(batch)
        probability = output['probability']
        p = tensor_to_mask(probability[0, 0])
        mask_Predict = cv2.resize(p, dsize=(image_size, image_size), interpolation=cv2.INTER_LINEAR)

    return mask_Predict


def get_learning_rate(optimizer):
    return optimizer.param_groups[0]['lr']


##数据处理
def img2tensor(img, dtype: np.dtype = np.float32):
    if img.ndim == 2: img = np.expand_dims(img, 2)
    img = np.transpose(img, (2, 0, 1))
    return torch.from_numpy(img.astype(dtype, copy=False))


def rle_encode_less_memory(img):
    # the image should be transposed
    pixels = img.T.flatten()
    # This simplified method requires first and last pixel to be zero
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]

    return ' '.join(str(x) for x in runs)


def valid_augment5(image, mask, organ=None):
    return image, mask
def Patch_validate(net, valid_loader):
    net = net.eval()
    image_size = int(valid_loader.dataset.df['img_height'][0])
    for t, batch in enumerate(valid_loader):
        net.output_type = ['inference']
        with torch.no_grad():
            with amp.autocast(enabled=is_amp):
                # batch['image'] = batch['image'].to(device)
                batch_list = batch['image'].detach().numpy()
                batch_list = np.squeeze(batch_list)
                batch_Image_ = batch_list.transpose(1, 2, 0)

                image_plt = batch_Image_
                image_plt = image_plt[:, :, ::-1]
                batch_list = do_image_cut(batch_Image_, 512)

                for index, image in enumerate(batch_list):
                    image = image[:, :, ::-1]
                    batch_list[index] = image.transpose(2, 0, 1)
                    batch_list[index] = np.ascontiguousarray(batch_list[index])
                batch['image'] = torch.tensor(batch_list, dtype=torch.float)
                batch['image'] = batch['image'].to(device)
                output = net(batch)

        probability = output['probability']

        # p = tensor_to_mask(probability[0, 0])
        p = probability.data.cpu().numpy()
        p = np.squeeze(p)
        mask_Predict = mask_merge(p, (image_plt.shape[0], image_plt.shape[1]), 512)


    return mask_Predict,batch_Image_
def Patch_valid_ImageShows(Data_Pd, model, imagesizes):
    ##读取数据
    valid_dataset = HubmapDataset(Data_Pd, valid_augment5, imagesizes)
    valid_dataset.fnames = Data_Pd['id'][0]
    valid_loader = DataLoader(
        valid_dataset,
        sampler=SequentialSampler(valid_dataset),
        batch_size=1,
        drop_last=False,
        num_workers=0,
        pin_memory=False,
        collate_fn=null_collate,
    )
    preds,Image_ = Patch_validate(model, valid_loader)
    del valid_dataset, valid_loader

    return preds,Image_

# Stochastic Weight Averaging
# https://pytorch.org/blog/pytorch-1.6-now-includes-stochastic-weight-averaging/
def do_swa(checkpoint):
    skip = ['relative_position_index', 'num_batches_tracked']
    
    K = len(checkpoint)
    swa = None
    
    for k in range(K):
        state_dict = torch.load(checkpoint[k], map_location=lambda storage, loc: storage)['state_dict']
        if swa is None:
            swa = state_dict
        else:
            for k, v in state_dict.items():
                #print(k)
                if any(s in k for s in skip): continue
                swa[k] += v
    
    for k, v in swa.items():
        if any(s in k for s in skip): continue
        swa[k] /= K
    
    return swa


def valid_ImageShows(Data_Pd,model,imagesizes):

    ##读取数据
    valid_dataset = HubmapDataset(Data_Pd, valid_augment5,imagesizes)
    valid_dataset.fnames = Data_Pd['id'][0]
    valid_loader = DataLoader(
        valid_dataset,
        sampler=SequentialSampler(valid_dataset),
        batch_size=1,
        drop_last=False,
        num_workers=0,
        pin_memory=False,
        collate_fn=null_collate,
    )
    preds = validate(model, valid_loader)
    del valid_dataset, valid_loader

    return preds
def ListModel_state(model_LD,state_dict_LD):
    model_LD.load_state_dict(state_dict_LD)
    model_LD.float()
    model_LD.eval()
    model_LD.to(device)
    
    
def swa_model_load(checkpoint, model_name="effnet"):
    
    swa = do_swa(checkpoint)
    torch.save({
        'state_dict': swa,
        'swa': [c.split('/')[-1] for c in checkpoint],
    }, 
        f'./{model_name}-swa.pth')

    effnet = EffUnet(config.model)
    state_dict = swa
    effnet.load_state_dict(state_dict, strict=False)  # True
    effnet.float()
    effnet.eval()
    effnet.to(device)
    return effnet



if __name__ == '__main__':

    #TH = 0.3 # threshold for positive predictions
    identity = rasterio.Affine(1, 0, 0, 0, 1, 0)
    DATA = '../input/hubmap-organ-segmentation/test_images/'
    df_sample = pd.read_csv('../input/hubmap-organ-segmentation/test.csv')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

 #Coat2_Fold0_10080  Coat2_Fold1_12390 Coat2_Fold2_19320
    Coat_Tensorform = ['../input/deeplearnings/Coat_Fold2_20370.model.pth',
                      '../input/deeplearnings/Coat2_Fold2_19320.model.pth']
    Frist_Coat_Tensorform=['../input/deeplearnings/Frist_Coat_Fold0_34860.model.pth']
    Y_Coat_Tensorform = ['../input/deeplearnings/Y_Coat_00010600_512X512.model.pth']
    
    #Coat2_Fold2_19320  Fold3_169830  PTV2_Fold0_20790  Coat2_Fold0_10080  Coat2_Fold1_12390 Coat2_Fold2_19320
    MY_Swin_Transform_Tensorform=['../input/deeplearnings/Fold3_169830_762and0.114.pth',
                                 '../input/deeplearnings/Swin_Fold1_15198.model.pth']
    #Coat_Tensorform=[]
    #PVT2_Model_Tensorform=['../input/deeplearnings/PVT2_Fold3_20790.model.pth']
    Swin_Transform_Tensorform=['../input/deeplearnings/Y_Swin_Transform_Fold_2_6912.model.pth',
                              '../input/deeplearnings/Y_Swin_Transform_Fold_3_6237.model.pth',
                              '../input/deeplearnings/Y_Swin_Transform_Fold_1_6912.model.pth']
    PVT2_Model_Tensorform=['../input/deeplearnings/PVT2/PTV2_Fold0_20790.model.pth',#IB=0.74
                           '../input/deeplearnings/PVT2_Fold-1_26880.model.pth',
                           '../input/deeplearnings/PVT2_Fold3_20790.model.pth',
                          '../input/deeplearnings/PVT2/PTV2_Fold2_18270.model.pth']
#Coat2_Fold1_12390 Coat2_Fold2_19320  Fold3_169830 PTV2_Fold0_20790 PVT2_Fold-1_26880  PVT2_Fold2_19110  Coat2_Fold0_10080  Coat2_Fold1_12390 Coat2_Fold2_19320
    
    Patch_Coat_Tensorform = ['../input/deeplearnings/Caot_Fold4_spleen00011610.model.pth',
                             '../input/deeplearnings/Caot_Fold4_largeintestine00009024.model.pth',
                             '../input/deeplearnings/Caot_Fold4_prostate00008550.model.pth',
                             '../input/deeplearnings/Caot_Fold4_kidney00009600.model.pth',
                            '../input/deeplearnings/Coat_Fold2_20370.model.pth']#'Coat_spleen00033024.model.pth
    
    valid = ['../input/hubmap-lungs-effnet-512-fold3-weights/00010600.model.pth',
            '../input/hubmap-lungs-effnet-512-fold3-weights/00010000.model.pth',
            '../input/hubmap-lungs-effnet-512-fold3-weights/00010700.model.pth']
    
    
    organ_Dict = {'spleen': 0, 'largeintestine': 1, 'prostate': 2,'kidney': 3,'lung': 4 }
    organs_lists = ['kidney', 'largeintestine', 'lung', 'prostate', 'spleen']
    names, preds = [], []
    for idx, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
        Data_Type=str(row['data_source'])
        idx = str(row['id'])
        organ_=str(row['organ'])
        data = rasterio.open(os.path.join(DATA, idx + '.tiff'),
                             transform=identity, num_threads='all_cpus')
        ##生成需要文件
        colname_ = ['id','organ', 'img_height', 'img_width']
        values = [idx,organ_,
                  data.meta['width'],
                  data.meta['height']]
        My_Submitfile = pd.DataFrame(np.array(values), index=np.array(colname_)).T
        mask = np.zeros((data.meta['width'], data.meta['width']), dtype=np.float32)
        
        
        ###Patch_Coat_Model
        #Patch_Coat_mask=np.zeros((data.meta['width'], data.meta['width']), dtype=np.float32)
        if organ_Dict[organ_]<=3:
            
            if Data_Type=='HPA':
                TH=0.3
            else:
                TH=0.3
            
            
            
            state_dict_Coat = torch.load(Patch_Coat_Tensorform[organ_Dict[organ_]], map_location=None)['state_dict']
            Patch_model_Coat = Coat_Net_0920()
            Patch_model_Coat = ListModel_state(Patch_model_Coat, state_dict_Coat)
            Patch_List_Image_Coat, Image_ = Patch_valid_ImageShows(Data_Pd=My_Submitfile, model=Patch_model_Coat,imagesizes=2022)
            if (data.meta['width'] != Patch_List_Image_Coat.shape[0]):
                Patch_List_Image_Coat = cv2.resize(Patch_List_Image_Coat, dsize=(data.meta['width'], data.meta['height']),
                                               interpolation=cv2.INTER_LINEAR)
            Patch_Coat_mask=Patch_List_Image_Coat
            
            Frist_Coat_mask = np.zeros((data.meta['width'], data.meta['width']), dtype=np.float32)
            for Coat_Transform_model in Frist_Coat_Tensorform:
                state_dict_Coat = torch.load(Coat_Transform_model, map_location=None)['state_dict']
                model_Coat = Net_Coat()
                model_Coat=ListModel_state(model_Coat,state_dict_Coat)
                List_Image_Coat=valid_ImageShows(Data_Pd=My_Submitfile, model=model_Coat,imagesizes=1024)
                Frist_Coat_mask+=List_Image_Coat
            Frist_Coat_mask=Frist_Coat_mask/len(Frist_Coat_Tensorform)
            
            
            
            PVT2_mask = np.zeros((data.meta['width'], data.meta['width']), dtype=np.float32)
            for pvt_model in PVT2_Model_Tensorform:
                state_dict_PVT2 = torch.load(pvt_model, map_location=None)['state_dict']
                model_PVT2 = Net_PVT2()
                model_PVT2 =ListModel_state(model_PVT2,state_dict_PVT2)
                List_Image_PVT2 = valid_ImageShows(Data_Pd=My_Submitfile, model=model_PVT2,imagesizes=768)
                PVT2_mask+=List_Image_PVT2
            PVT2_mask=PVT2_mask/len(PVT2_Model_Tensorform)
            
            
            My_Swin_mask=np.zeros((data.meta['width'], data.meta['width']), dtype=np.float32)
            for Swin_Transform_model in MY_Swin_Transform_Tensorform:
                state_dict_Swin_Transform = torch.load(Swin_Transform_model, map_location=None)['state_dict']
                model_Swin_Transform = Net_Swin_Transform()
                model_Swin_Transform =ListModel_state(model_Swin_Transform,state_dict_Swin_Transform)
                List_Image_Swin_Transform = valid_ImageShows(Data_Pd=My_Submitfile, model=model_Swin_Transform,imagesizes=768)
                My_Swin_mask+=List_Image_Swin_Transform
            My_Swin_mask=My_Swin_mask/len(MY_Swin_Transform_Tensorform)
#             PVT2_mask=(PVT2_mask+My_Swin_mask)/2
            
            
            Swin_mask=np.zeros((data.meta['width'], data.meta['width']), dtype=np.float32)
            for Swin_Transform_model in Swin_Transform_Tensorform:
                state_dict_Swin_Transform = torch.load(Swin_Transform_model, map_location=None)['state_dict']
                model_Swin_Transform = Net_Swin_Transform()
                model_Swin_Transform =ListModel_state(model_Swin_Transform,state_dict_Swin_Transform)
                List_Image_Swin_Transform = valid_ImageShows(Data_Pd=My_Submitfile, model=model_Swin_Transform,imagesizes=768)
                Swin_mask+=List_Image_Swin_Transform
            Swin_mask=Swin_mask/len(Swin_Transform_Tensorform)
            
            Coat_mask=np.zeros((data.meta['width'], data.meta['width']), dtype=np.float32)
            ###Coat_Model
            for Coat_Transform_model in Coat_Tensorform:
                state_dict_Coat = torch.load(Coat_Transform_model, map_location=None)['state_dict']
                model_Coat = Coat_Net2()
                model_Coat=ListModel_state(model_Coat,state_dict_Coat)
                List_Image_Coat=valid_ImageShows(Data_Pd=My_Submitfile, model=model_Coat,imagesizes=1024)
                Coat_mask+=List_Image_Coat
            Coat_mask=Coat_mask/len(Coat_Tensorform)
            
            Patch_Coat_mask=(PVT2_mask+Patch_Coat_mask+Swin_mask+Coat_mask+My_Swin_mask+Frist_Coat_mask)/6

            #TH=0.3
            mask=Patch_Coat_mask
            mask[mask > TH] = 1
            mask[mask < 1] = 0
            preds.append(rle_encode_less_memory(mask))
            names.append(int(idx))
            
        else:

            
            Coat_mask=np.zeros((data.meta['width'], data.meta['width']), dtype=np.float32)
            ###Coat_Model
            for Coat_Transform_model in Coat_Tensorform:
                state_dict_Coat = torch.load(Coat_Transform_model, map_location=None)['state_dict']
                model_Coat = Coat_Net2()
                model_Coat=ListModel_state(model_Coat,state_dict_Coat)
                List_Image_Coat=valid_ImageShows(Data_Pd=My_Submitfile, model=model_Coat,imagesizes=1024)
                Coat_mask+=List_Image_Coat
            Coat_mask=Coat_mask/len(Coat_Tensorform)
            My_Swin_mask=np.zeros((data.meta['width'], data.meta['width']), dtype=np.float32)
            for Swin_Transform_model in MY_Swin_Transform_Tensorform:
                state_dict_Swin_Transform = torch.load(Swin_Transform_model, map_location=None)['state_dict']
                model_Swin_Transform = Net_Swin_Transform()
                model_Swin_Transform =ListModel_state(model_Swin_Transform,state_dict_Swin_Transform)
                List_Image_Swin_Transform = valid_ImageShows(Data_Pd=My_Submitfile, model=model_Swin_Transform,imagesizes=768)
                My_Swin_mask+=List_Image_Swin_Transform
            My_Swin_mask=My_Swin_mask/len(MY_Swin_Transform_Tensorform)
            Frist_Coat_mask = np.zeros((data.meta['width'], data.meta['width']), dtype=np.float32)
            for Coat_Transform_model in Frist_Coat_Tensorform:
                state_dict_Coat = torch.load(Coat_Transform_model, map_location=None)['state_dict']
                model_Coat = Net_Coat()
                model_Coat=ListModel_state(model_Coat,state_dict_Coat)
                List_Image_Coat=valid_ImageShows(Data_Pd=My_Submitfile, model=model_Coat,imagesizes=1024)
                Frist_Coat_mask+=List_Image_Coat
            Frist_Coat_mask=Frist_Coat_mask/len(Frist_Coat_Tensorform)
            
            PVT2_mask = np.zeros((data.meta['width'], data.meta['width']), dtype=np.float32)
            for pvt_model in PVT2_Model_Tensorform:
                state_dict_PVT2 = torch.load(pvt_model, map_location=None)['state_dict']
                model_PVT2 = Net_PVT2()
                model_PVT2 =ListModel_state(model_PVT2,state_dict_PVT2)
                List_Image_PVT2 = valid_ImageShows(Data_Pd=My_Submitfile, model=model_PVT2,imagesizes=768)
                PVT2_mask+=List_Image_PVT2
            PVT2_mask=PVT2_mask/len(PVT2_Model_Tensorform)
            
            SWA_Model = swa_model_load(valid, model_name="effnet")
            swa_lung_model_pred = valid_ImageShows(Data_Pd=My_Submitfile, model=SWA_Model,imagesizes=512)
            
            
            Swin_mask=np.zeros((data.meta['width'], data.meta['width']), dtype=np.float32)
            for Swin_Transform_model in Swin_Transform_Tensorform:
                state_dict_Swin_Transform = torch.load(Swin_Transform_model, map_location=None)['state_dict']
                model_Swin_Transform = Net_Swin_Transform()
                model_Swin_Transform =ListModel_state(model_Swin_Transform,state_dict_Swin_Transform)
                List_Image_Swin_Transform = valid_ImageShows(Data_Pd=My_Submitfile, model=model_Swin_Transform,imagesizes=768)
                Swin_mask+=List_Image_Swin_Transform
            Swin_mask=Swin_mask/len(Swin_Transform_Tensorform)
            Patch_Coat_mask=(Swin_mask+PVT2_mask+Coat_mask+My_Swin_mask+Frist_Coat_mask+swa_lung_model_pred)/6
            TH=0.1
            mask=Patch_Coat_mask
            mask[mask > TH] = 1
            mask[mask < 1] = 0
            preds.append(rle_encode_less_memory(mask))
            names.append(int(idx))
       


        del data, My_Submitfile, mask,Patch_Coat_mask#,Coat_mask,Swin_mask,PVT2_mask
        gc.collect()
    df = pd.DataFrame({'id': names, 'rle': preds})
    df.to_csv('submission.csv', index=False)

  0%|          | 0/1 [00:07<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'eval'